In [1]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", force_download=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [2]:
print(tokenizer.special_tokens_map)


{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


In [3]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Test text
texts = ["Short sentence.", "A much longer sentence for testing purposes."]

# Tokenize with padding
encoded = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
print(encoded)


{'input_ids': tensor([[ 101, 2460, 6251, 1012,  102,    0,    0,    0,    0,    0],
        [ 101, 1037, 2172, 2936, 6251, 2005, 5604, 5682, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
from span_marker import SpanMarkerModel

# Load the model
from span_marker import SpanMarkerModel

# Load the model
model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")

# Check and manually set pad_token if missing
if not hasattr(model.tokenizer, "pad_token") or model.tokenizer.pad_token is None:
    print("Setting pad_token manually...")
    model.tokenizer.pad_token = "[PAD]"
    model.tokenizer.pad_token_id = model.tokenizer.convert_tokens_to_ids("[PAD]")

# Verify pad_token and pad_token_id
print("Pad token:", model.tokenizer.pad_token)
print("Pad token ID:", model.tokenizer.pad_token_id)

model.predict("Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris.")  # The model expects a list of texts
   

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Setting pad_token manually...
Pad token: [PAD]
Pad token ID: 0


[{'span': 'Amelia Earhart',
  'label': 'PER',
  'score': 0.9999709129333496,
  'char_start_index': 0,
  'char_end_index': 14},
 {'span': 'Lockheed Vega 5B',
  'label': 'VEHI',
  'score': 0.9050098657608032,
  'char_start_index': 38,
  'char_end_index': 54},
 {'span': 'Atlantic',
  'label': 'LOC',
  'score': 0.9991973042488098,
  'char_start_index': 66,
  'char_end_index': 74},
 {'span': 'Paris',
  'label': 'LOC',
  'score': 0.9999232292175293,
  'char_start_index': 78,
  'char_end_index': 83}]

In [5]:
def load_entities_mapping(file_path):
    """
    Loads entities and their translations from the given JSON file, skipping invalid lines.
    Args:
        file_path (str): Path to the JSON file containing entity mappings.
    Returns:
        list: A list of dictionaries representing the JSON data.
    """
    with open(file_path, 'r') as jf:
        try:
            return json.load(jf)  # Load the entire JSON array
        except json.JSONDecodeError as e:
            print(f"Error loading JSON file: {e}")
            return []


def mask_entities_with_spanmarker(text, model):
    """
    Masks named entities in text using predictions from SpanMarkerModel.

    Args:
        text (Union[str, List[str]]): Input text(s). Can be a single string or a list of strings.
        model (SpanMarkerModel): Pretrained SpanMarker model for entity prediction.

    Returns:
        Union[Tuple[str, dict], List[Tuple[str, dict]]]:
            - For a single string:
                - masked_text (str): Text with entities replaced by placeholders.
                - mapping (dict): Mapping of placeholders to original entities.
            - For a list of strings:
                - List of (masked_text, mapping) tuples.
    """
    # Check if input is a single string or a list of strings
    is_single_string = isinstance(text, str)
    if is_single_string:
        text = [text]  # Convert single string to list for uniform processing

    # Perform predictions using the model
    predictions = model.predict(text)  # Handles batch processing

    results = []
    for i, sentence in enumerate(text):
        masked_text = sentence
        mapping = {}
        type_counters = {}
        offset = 0

        # If no predictions for this sentence
        if not predictions[i]:  # Empty list
            print(f"No entities found for text: {sentence}")
            results.append((masked_text, mapping))
            continue

        # Process each entity in the prediction
        for entity in predictions[i]:
            entity_text = entity['span']
            entity_type = entity['label'].upper()
            start = entity['char_start_index']
            end = entity['char_end_index']

            # Counter for entity type
            if entity_type not in type_counters:
                type_counters[entity_type] = 1
            else:
                type_counters[entity_type] += 1

            # Generate a placeholder
            placeholder = f"[ENTITY_{entity_type}_{type_counters[entity_type]}]"

            # Replace entity with placeholder in the text
            start += offset
            end += offset
            masked_text = masked_text[:start] + placeholder + masked_text[end:]
            offset += len(placeholder) - len(entity_text)

            # Add to mapping
            mapping[placeholder] = {
                "original_text": entity_text,
                "type": entity_type,
                "confidence": entity['score'],
                "start": start,
                "end": end
            }

        results.append((masked_text, mapping))

    # Return a single result if input was a single string, else return the list
    return results[0] if is_single_string else results



In [6]:
import json
italiantrain = "/Users/inaocelli/Documents/CLASSES FALL 2024/CSCI 375/Final/EAMT/data/italian_train.json"  # Update with the correct path

entity_data = load_entities_mapping(italiantrain)

# Test loading result
if entity_data:
    eng_text = entity_data[4]['source']  # English text
    itl_text = entity_data[4]['target']  # Italian text
    print("English Example:", eng_text)
    print("Italian Example:", itl_text)
else:
    print("No valid data loaded.")

English Example: Is Lake Baikal located in Russia?
Italian Example: Il lago Baikal è situato in Russia?


In [7]:
italiantrain = "/Users/inaocelli/Documents/CLASSES FALL 2024/CSCI 375/Final/EAMT/data/italian_train.json"  # Update with the correct path

def ensure_pad_token(model):
    if not hasattr(model.tokenizer, "pad_token") or model.tokenizer.pad_token is None:
        print("Setting pad_token manually...")
        model.tokenizer.pad_token = "[PAD]"
        model.tokenizer.pad_token_id = model.tokenizer.convert_tokens_to_ids("[PAD]")

        
# Load the dataset
entity_data = load_entities_mapping(italiantrain)

# Test loading result
if entity_data:
    eng_text = entity_data[12]['source']  # English text
    itl_text = entity_data[12]['target']  # Italian text
    print("English Example:", eng_text)
    print("Italian Example:", itl_text)

    # Load the model
    model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")
    ensure_pad_token(model)

    # Call the function and print results
    masked_eng_text, mapping_eng = mask_entities_with_spanmarker(eng_text, model)
    masked_itl_text, mapping_itl = mask_entities_with_spanmarker(itl_text, model)

    print("\nOriginal English Text:", eng_text)
    print("Masked English Text:", masked_eng_text)
    print("Mapping (English):", json.dumps(mapping_eng, indent=4))

    print("\nOriginal Italian Text:", itl_text)
    print("Masked Italian Text:", masked_itl_text)
    print("Mapping (Italian):", json.dumps(mapping_itl, indent=4))
else:
    print("No valid data loaded.")

English Example: Which pharaoh of Egypt was the youngest?
Italian Example: Chi fu il più giovane faraone d'Egitto?
Setting pad_token manually...

Original English Text: Which pharaoh of Egypt was the youngest?
Masked English Text: Which pharaoh of [ENTITY_LOC_1] was the youngest?
Mapping (English): {
    "[ENTITY_LOC_1]": {
        "original_text": "Egypt",
        "type": "LOC",
        "confidence": 0.9993298053741455,
        "start": 17,
        "end": 22
    }
}

Original Italian Text: Chi fu il più giovane faraone d'Egitto?
Masked Italian Text: Chi fu il più giovane faraone d'[ENTITY_LOC_1]?
Mapping (Italian): {
    "[ENTITY_LOC_1]": {
        "original_text": "Egitto",
        "type": "LOC",
        "confidence": 0.9980237483978271,
        "start": 32,
        "end": 38
    }
}


In [8]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use mps:0


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the M2M100 model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

def translate_with_placeholders_m2m(masked_text, mapping, src_lang, tgt_lang):
    """
    Translates the non-entity parts of a text and re-inserts placeholders using M2M100.

    Args:
        masked_text (str): Text with placeholders for entities.
        mapping (dict): Mapping of placeholders to entity details.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translated text with placeholders intact.
    """
    tokenizer.src_lang = src_lang

    # Encode and translate the entire sentence
    encoded_text = tokenizer(masked_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang),
        max_length=128,  # Ensure sufficient length for translation
        early_stopping=True
    )

    # Decode the translated sentence
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
# Translate
src_lang = "en"
tgt_lang = "it"
masked_eng_text = "Which pharaoh of [ENTITY_LOC_1] was the youngest?"
translated_text = translate_with_placeholders_m2m(masked_eng_text, mapping_eng, src_lang, tgt_lang)

# Print the result
print("Translated Text:", translated_text)

Translated Text: Quale faraone di [ENTITY_LOC_1] era il più giovane?


In [12]:
pip install sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Example: Translate an entity (if needed) based on the context
def translate_entity(entity, src_lang, tgt_lang):
    """
    Translates an entity if its type requires translation, otherwise returns the original entity.

    Args:
        entity (str): The placeholder or entity to process.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: The translated or original entity.
    """
    # types that should not be translated
    non_translatable_types = {"PER", "INST", "MEDIA", "PLANT", "VEHI"}

    # Check if the entity contains one of the non-translatable types
    if any(ntype in entity for ntype in non_translatable_types):
        return entity 

    # Translate the entity using M2M100
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    tokenizer.src_lang = src_lang
    encoded_entity = tokenizer(entity, return_tensors="pt")
    generated_tokens = model.generate(**encoded_entity, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    translated_entity = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_entity



def reintegrate_entities(translation, mapping, src_lang, tgt_lang):
    """
    Replaces placeholders in the translated text with original or translated entities.

    Args:
        translation (str): Translated text with placeholders.
        mapping (dict): Mapping of placeholders to original entities.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translation with entities reintegrated.
    """
    for placeholder, entity_info in mapping.items():
        # Get the original entity
        original_entity = entity_info["original_text"]

        # Translate the entity if needed
        translated_entity = translate_entity(original_entity, src_lang, tgt_lang)

        # Replace placeholder with the translated entity
        translation = translation.replace(placeholder, translated_entity)

    return translation


In [22]:
# Example masked text and mapping
masked_eng_text = "Who is older, [ENTITY_PERSON-ARTIST/AUTHOR_1] or [ENTITY_PERSON-ARTIST/AUTHOR_2]?"
mapping_eng = {
    "[ENTITY_PERSON-ARTIST/AUTHOR_1]": {
        "original_text": "Paul McCartney",
        "type": "PERSON-ARTIST/AUTHOR",
        "confidence": 0.6339015364646912,
        "start": 14,
        "end": 28
    },
    "[ENTITY_PERSON-ARTIST/AUTHOR_2]": {
        "original_text": "Keith Richards",
        "type": "PERSON-ARTIST/AUTHOR",
        "confidence": 0.5666552782058716,
        "start": 49,
        "end": 63
    }
}

# Translate the masked text
src_lang = "en"
tgt_lang = "it"
translated_masked_text = "Chi è più [ENTITY_PERSON-ARTIST/AUTHOR_1] vecchio, o [ENTITY_PERSON-ARTIST/AUTHOR_2]?"

# Reintegration of entities
final_translation = reintegrate_entities(translated_masked_text, mapping_eng, src_lang, tgt_lang)

# Print the final translation
print("Final Translation:", final_translation)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Chi è più di Paul McCartney vecchio, o di Keith Richards?


In [23]:
import os
it_datapath = os.path.abspath('data/italian_train.json')

    # Step 1: Load the data
print("Loading Italian data...")
italian_data = load_entities_mapping(it_datapath)


Loading Italian data...


In [24]:
italian_data

[{'id': '8c6e2fc7',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Who is older, Paul McCartney or Keith Richards?',
  'target': 'Chi è più grande, Paul Mc Cartney o Keith Richards?',
  'entities': {'Q2599': {'it': 'Paul McCartney', 'en': 'Paul McCartney'}},
  'from': 'mintaka'},
 {'id': '4aaa6f8f',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'How many Grammy Awards did Jay-Z win in 2021?',
  'target': 'Quanti Grammy Awards ha vinto Jay Z nel 2021?',
  'entities': {'Q62766': {'it': 'Jay-Z', 'en': 'Jay-Z'}},
  'from': 'mintaka'},
 {'id': '1ab80cb2',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'What is the third installment of the Super Smash Bros. series?',
  'target': 'Qual è il terzo capitolo della serie di Super Smash Bros?',
  'entities': {'Q2778220': {'it': 'Super Smash Bros.',
    'en': 'Super Smash Bros.'}},
  'from': 'mintaka'},
 {'id': 'be0c4040',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Did the Axis win 

In [25]:
print("Loading SpanMarker model...")
spanmarker_model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")
ensure_pad_token(spanmarker_model)

Loading SpanMarker model...
Setting pad_token manually...


In [26]:
print("Loading M2M100 translation model...")
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

Loading M2M100 translation model...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
translated_sentences = []
max_entries = 10

for idx, entry in enumerate(italian_data[:max_entries]):  # Process only the first 10 entries
    print(f"Processing entry {idx + 1}/{max_entries}...")

    # Get source text (English) and target text (Italian)
    eng_text = entry['source']
    itl_text = entry['target']

    # Mask entities in the English text
    masked_eng_text, mapping = mask_entities_with_spanmarker(eng_text, spanmarker_model)
    print(f"Masked English Text: {masked_eng_text}")
    print(f"Entity Mapping: {json.dumps(mapping, indent=4)}")

    # Translate non-entity parts of the text
    m2m_tokenizer.src_lang = "en"
    tgt_lang = "it"
    translated_text_with_placeholders = translate_with_placeholders_m2m(
        masked_eng_text, mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Translated Text with Placeholders: {translated_text_with_placeholders}")

    # Reintegrate entities into the translated text
    final_translation = reintegrate_entities(
        translated_text_with_placeholders, mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Final Translation: {final_translation}")

    # Append final translation to the results
    translated_sentences.append({
        "id": entry["id"],
        "source": eng_text,
        "target": itl_text,
        "masked_source": masked_eng_text,
        "translation": final_translation,
        "mapping": mapping
    })

# Step 5: Save results
results_path = "data/translated_italian_first10.json"
with open(results_path, "w") as results_file:
    json.dump(translated_sentences, results_file, indent=4)

print(f"Translation results for first 10 entries saved to {results_path}")

Processing entry 1/10...
Masked English Text: Who is older, [ENTITY_PER_1] or [ENTITY_PER_2]?
Entity Mapping: {
    "[ENTITY_PER_1]": {
        "original_text": "Paul McCartney",
        "type": "PER",
        "confidence": 0.9999871253967285,
        "start": 14,
        "end": 28
    },
    "[ENTITY_PER_2]": {
        "original_text": "Keith Richards",
        "type": "PER",
        "confidence": 0.9999836683273315,
        "start": 32,
        "end": 46
    }
}
Translated Text with Placeholders: Chi è più vecchio, [ENTITY_PER_1] o [ENTITY_PER_2]?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Chi è più vecchio, di Paul McCartney o di Keith Richards?
Processing entry 2/10...
Masked English Text: How many Grammy Awards did [ENTITY_PER_1] win in 2021?
Entity Mapping: {
    "[ENTITY_PER_1]": {
        "original_text": "Jay-Z",
        "type": "PER",
        "confidence": 0.9989475607872009,
        "start": 27,
        "end": 32
    }
}
Translated Text with Placeholders: Quanti Grammy Awards hanno vinto [ENTITY_PER_1] nel 2021?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quanti Grammy Awards hanno vinto di Jay-Z nel 2021?
Processing entry 3/10...
Masked English Text: What is the third installment of the [ENTITY_MEDIA_1] series?
Entity Mapping: {
    "[ENTITY_MEDIA_1]": {
        "original_text": "Super Smash Bros.",
        "type": "MEDIA",
        "confidence": 0.9993467926979065,
        "start": 37,
        "end": 54
    }
}
Translated Text with Placeholders: Qual è la terza installazione della serie [ENTITY_MEDIA_1]?


...

Final Translation: Qual è la terza installazione della serie di Super Smash Bros.?
Processing entry 4/10...
Masked English Text: Did the [ENTITY_ORG_1] win [ENTITY_EVE_1]?
Entity Mapping: {
    "[ENTITY_ORG_1]": {
        "original_text": "Axis",
        "type": "ORG",
        "confidence": 0.7516891360282898,
        "start": 8,
        "end": 12
    },
    "[ENTITY_EVE_1]": {
        "original_text": "World War 1",
        "type": "EVE",
        "confidence": 0.9989414811134338,
        "start": 27,
        "end": 38
    }
}
Translated Text with Placeholders: Il vincitore [ENTITY_ORG_1] ha vinto [ENTITY_EVE_1]?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Il vincitore Assi ha vinto Guerra Mondiale 1?
Processing entry 5/10...
Masked English Text: Is [ENTITY_LOC_1] located in [ENTITY_LOC_2]?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "Lake Baikal",
        "type": "LOC",
        "confidence": 0.9999220371246338,
        "start": 3,
        "end": 14
    },
    "[ENTITY_LOC_2]": {
        "original_text": "Russia",
        "type": "LOC",
        "confidence": 0.9999232292175293,
        "start": 29,
        "end": 35
    }
}
Translated Text with Placeholders: È [ENTITY_LOC_1] situato in [ENTITY_LOC_2]?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: È Il lago Baikal situato in La Russia?
Processing entry 6/10...
Masked English Text: Which country has a larger area, [ENTITY_LOC_1] or [ENTITY_LOC_2]?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "Canada",
        "type": "LOC",
        "confidence": 0.9999555349349976,
        "start": 33,
        "end": 39
    },
    "[ENTITY_LOC_2]": {
        "original_text": "Mexico",
        "type": "LOC",
        "confidence": 0.9999406337738037,
        "start": 51,
        "end": 57
    }
}


In [20]:
import math
from collections import Counter

def ngrams(sentence, n):
    """Generate n-grams from a given sentence (list of tokens)."""
    return [tuple(sentence[i:i+n]) for i in range(len(sentence)-n+1)]

def clipped_counts(candidate, references, n):
    """
    Compute clipped n-gram counts for a given n.
    Returns the total clipped count and the total number of candidate n-grams.
    """
    candidate_ngrams = Counter(ngrams(candidate, n))
    max_ref_counts = {}

    for ref in references:
        ref_counts = Counter(ngrams(ref, n))
        for ng in ref_counts:
            if ng not in max_ref_counts:
                max_ref_counts[ng] = ref_counts[ng]
            else:
                max_ref_counts[ng] = max(max_ref_counts[ng], ref_counts[ng])

    clipped = 0
    for ng in candidate_ngrams:
        # Clipped count is min(candidate_count, max_ref_count for that n-gram)
        clipped += min(candidate_ngrams[ng], max_ref_counts.get(ng, 0))
    total = sum(candidate_ngrams.values())
    return clipped, total

def brevity_penalty(candidate, references):
    """
    Compute the brevity penalty.
    Uses the reference length closest to candidate length.
    """
    c = len(candidate)
    ref_lengths = [len(ref) for ref in references]
    # Find reference length closest to c
    closest_ref_len = min(ref_lengths, key=lambda r: (abs(r - c), r))
    
    if c > closest_ref_len:
        return 1.0
    else:
        return math.exp(1 - float(closest_ref_len) / c) if c > 0 else 0.0

def bleu_score(candidate, references, max_n=4):
    """
    Compute BLEU score for a candidate against one or more references.
    """
    # Calculate n-gram precisions
    precisions = []
    for n in range(1, max_n+1):
        clipped, total = clipped_counts(candidate, references, n)
        if total > 0:
            precisions.append(clipped / total)
        else:
            # If no n-grams of this order exist in the candidate, precision is 0
            precisions.append(0.0)

    # If all precisions are zero, BLEU is zero
    if all(p == 0 for p in precisions):
        return 0.0

    # Geometric mean of precisions
    log_precisions = [math.log(p) for p in precisions if p > 0]
    geo_mean = math.exp(sum(log_precisions) / len(log_precisions)) if len(log_precisions) > 0 else 0.0

    # Brevity penalty
    bp = brevity_penalty(candidate, references)

    # BLEU
    bleu = bp * geo_mean
    return bleu

# Example usage:
candidate = "the cat is on the mat".split()
references = [
    "the cat is sitting on the mat".split(),
    "there is a cat on the mat".split()
]

score = bleu_score(candidate, references)
print("BLEU score:", score)


BLEU score: 0.6236930675040936
